In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import jellyfish
import snowflake.connector
import time

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import inaugural
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

import gensim
from gensim.utils import simple_preprocess
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import num2words
from num2words import num2words

pd.options.display.max_colwidth = 1000

/home/rkothari/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
start = time.time()

In [3]:
data = pd.read_csv('~/Downloads/invoice_product_descriprions',sep=',')

In [4]:
res = data[:1000]

In [5]:
def cleaned_text(text):
    text = text.replace("/", " by ")
    text.split(" ")
   
    text = ''.join(num2words(word) if word.isdigit() else word for word in text  )
    
    word = nltk.tokenize.word_tokenize(text)
    return(text)

In [6]:
res.columns

Index(['Unnamed: 0', 'GTIN', 'INVOICE_PACKAGE_DESCRIPTION',
       'PRODUCT_DESCRIPTION', 'INVOICE_UPC_CASE'],
      dtype='object')

In [7]:
#res['tokeized_invoice'] = res.apply(lambda x: cleaned_text(x.INVOICE_PACKAGE_DESCRIPTION), axis=1) 
res['tokeized_product'] = res.apply(lambda x: cleaned_text(x.PRODUCT_DESCRIPTION), axis=1) 


In [8]:
res['tokeized_invoice'] = res.apply(lambda x: cleaned_text(x.INVOICE_PACKAGE_DESCRIPTION), axis=1)

In [9]:
res['TOKENIZED_INVOICE_SOUNDEX'] = res.apply(lambda x: jellyfish.soundex(x.tokeized_invoice), axis=1) 

In [10]:
res['TOKENIZED_PRODUCT_DESCRIPTION_SOUNDEX'] = res.apply(lambda x: jellyfish.soundex(x.tokeized_product), axis=1) 

In [11]:
res['INVOICE_SOUNDEX'] = res.apply(lambda x: jellyfish.soundex(x.INVOICE_PACKAGE_DESCRIPTION), axis=1) 

In [12]:
res['PRODUCT_DESCRIPTION_SOUNDEX'] = res.apply(lambda x: jellyfish.soundex(x.PRODUCT_DESCRIPTION), axis=1) 

In [13]:
res['jellyfish_ratio'] = res.apply(lambda x: fuzz.partial_ratio(x.INVOICE_SOUNDEX, x.PRODUCT_DESCRIPTION_SOUNDEX), axis=1)


In [14]:
res['tokenized_jellyfish_ratio'] = res.apply(lambda x: fuzz.partial_ratio(x.TOKENIZED_INVOICE_SOUNDEX, x.TOKENIZED_PRODUCT_DESCRIPTION_SOUNDEX), axis=1)


In [15]:
end = time.time()

In [16]:
print("time = ", (end-start))

time =  5.0035011768341064


In [17]:
res.to_csv("~/Downloads/ratios_of_tokenized_soundex.csv")

# Approach 2

In [18]:
data2 = pd.read_csv('~/Downloads/invoice_product_descriprions',sep=',')

In [19]:
res2 = data2[:100000]

In [20]:
res2

,Unnamed: 0,GTIN,INVOICE_PACKAGE_DESCRIPTION,PRODUCT_DESCRIPTION,INVOICE_UPC_CASE
0,0,1820011047,BUDWEISER 2/12 CN 12OZ,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047
1,1,1820011047,BUD 16OZ 4/6 CAN,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047
2,2,1820011047,BUD 2/12 CAN,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047
3,3,1820011047,BUDWEISER 12PK CANS PC:02/10/20 12 O,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047
4,4,1820011047,BUD 12 PK CANS,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047
...,...,...,...,...,...
99995,99995,33544000434,CORONA SELTZ VP 2/12 CAN,CORONA HARD SELTZER VARIETY PACK MALT BEVERAGE 12 CT 12 OZ CAN IN BOX 144 OZ,33544000434
99996,99996,33544000434,CORONA SELTZER 2/12/12 CN,CORONA HARD SELTZER VARIETY PACK MALT BEVERAGE 12 CT 12 OZ CAN IN BOX 144 OZ,33544000434
99997,99997,33544000434,CORONA SELTZER VAR 2/12,CORONA HARD SELTZER VARIETY PACK MALT BEVERAGE 12 CT 12 OZ CAN IN BOX 144 OZ,33544000434
99998,99998,33544000434,CORONA SELTZER VARIETY 2/12PK CANS,CORONA HARD SELTZER VARIETY PACK MALT BEVERAGE 12 CT 12 OZ CAN IN BOX 144 OZ,3354400043


In [21]:
# r'(\d\s+CT\s+\d\s+OZ)'

In [22]:
rgx_list = [r'\s+\d+\s+CT\s+\d+\s+OZ', r'\d+\s+OZ', r'\s+\d+/\d+',r'\s+\d+OZ', r'CANS', r'CAN', r'OZ', r'BOX' , r'IN',r'PK', r'CN', r'\d+', r'CASE',
           r'\d+L/d+\s+PET',r'\.',r'\.L',r'NR', r'/',r'O', r'()',r'//','LTR', r'PET',r'ML',r'\?',r'BBL']


In [23]:
import re
def remove_pattern(rgx_list, text):
    for r in rgx_list:
        
        text = re.sub(r, '', text )
        
    return text


In [24]:
res2['clean_product_desc'] = res2.apply(lambda x: remove_pattern(rgx_list,x.PRODUCT_DESCRIPTION), axis=1) 
res2['clean_invoice_desc'] = res2.apply(lambda x: remove_pattern(rgx_list,x.INVOICE_PACKAGE_DESCRIPTION), axis=1) 


In [25]:
res2.head()

,Unnamed: 0,GTIN,INVOICE_PACKAGE_DESCRIPTION,PRODUCT_DESCRIPTION,INVOICE_UPC_CASE,clean_product_desc,clean_invoice_desc
0,0,1820011047,BUDWEISER 2/12 CN 12OZ,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047,BUDWEISER BEER,BUDWEISER
1,1,1820011047,BUD 16OZ 4/6 CAN,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047,BUDWEISER BEER,BUD
2,2,1820011047,BUD 2/12 CAN,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047,BUDWEISER BEER,BUD
3,3,1820011047,BUDWEISER 12PK CANS PC:02/10/20 12 O,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047,BUDWEISER BEER,BUDWEISER PC:
4,4,1820011047,BUD 12 PK CANS,BUDWEISER BEER 12 CT 12 OZ CAN IN BOX 144 OZ,1820011047,BUDWEISER BEER,BUD


In [26]:
res2['SOUNDEX_clean_invoice'] = res2.apply(lambda x: jellyfish.soundex(x.clean_invoice_desc), axis=1) 


In [27]:
res2['SOUNDEX_clean_product'] = res2.apply(lambda x: jellyfish.soundex(x.clean_product_desc), axis=1) 

In [28]:
res2['jellyfish_ratio'] = res2.apply(lambda x: fuzz.partial_ratio(x.SOUNDEX_clean_invoice, x.SOUNDEX_clean_product), axis=1)
res2['fuzz_ratio'] = res2.apply(lambda x: fuzz.partial_ratio(x.clean_invoice_desc, x.clean_product_desc), axis=1)


In [29]:
res2[['clean_invoice_desc','clean_product_desc','jellyfish_ratio','fuzz_ratio']]

,clean_invoice_desc,clean_product_desc,jellyfish_ratio,fuzz_ratio
0,BUDWEISER,BUDWEISER BEER,100,100
1,BUD,BUDWEISER BEER,50,75
2,BUD,BUDWEISER BEER,50,75
3,BUDWEISER PC:,BUDWEISER BEER,100,78
4,BUD,BUDWEISER BEER,50,50
...,...,...,...,...
99995,CRNA SELTZ VP,CRNA HARD SELTZER VARIETY PACK MALT BEVERAGE,75,64
99996,CRNA SELTZER,CRNA HARD SELTZER VARIETY PACK MALT BEVERAGE,75,77
99997,CRNA SELTZER VAR,CRNA HARD SELTZER VARIETY PACK MALT BEVERAGE,75,81
99998,CRNA SELTZER VARIETY,CRNA HARD SELTZER VARIETY PACK MALT BEVERAGE,75,86


In [30]:
res2.to_csv('~/Downloads/ratios_of_cleaned_desc.csv')

In [31]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
tokenizer = CountVectorizer(strip_accents='unicode',analyzer='char_wb',ngram_range=(3,3))

In [33]:
featurenames_product_desc = []
sparse_products_desc = []

featurenamess_invocie_desc = []
sparse_invoice_desc = []

In [34]:
res2.columns

Index(['Unnamed: 0', 'GTIN', 'INVOICE_PACKAGE_DESCRIPTION',
       'PRODUCT_DESCRIPTION', 'INVOICE_UPC_CASE', 'clean_product_desc',
       'clean_invoice_desc', 'SOUNDEX_clean_invoice', 'SOUNDEX_clean_product',
       'jellyfish_ratio', 'fuzz_ratio'],
      dtype='object')

In [35]:
res2["clean_invoice_desc"]

0                   BUDWEISER 
1                         BUD 
2                         BUD 
3         BUDWEISER     PC:   
4                       BUD   
                 ...          
99995           CRNA SELTZ VP 
99996            CRNA SELTZER 
99997         CRNA SELTZER VAR
99998    CRNA SELTZER VARIETY 
99999            CRNA SELTZER 
Name: clean_invoice_desc, Length: 100000, dtype: object

In [39]:

 

for i in res2["clean_invoice_desc"]:

    x = tokenizer.fit_transform([i])
    sparse_invoice_desc.append(x.toarray())
    featurenamess_invocie_desc.append(tokenizer.get_feature_names())
 


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [38]:
res2["featurenames_product_desc"] =  featurenamess_invocie_desc

In [ ]:
res2.columns

In [ ]:
card_docs = [TaggedDocument(row, [l]) for l, row in enumerate(featurenames)]
model = Doc2Vec(vector_size=50, min_count=0, epochs=100, dm=1)
model.build_vocab(card_docs)
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)


In [ ]:

ranks = []
for i in mapped01.itertuples():
    inferred_vector = model.infer_vector(i.INVOICE_PACKAGE_DESCRIPTION.split(' '))
    sims = model.wv.most_similar([inferred_vector], topn=len(card2vec))
    print('Tested Line: ', i.INVOICE_PACKAGE_DESCRIPTION)
    print(sims)